In [37]:
#Imports of libraries, pandas options

import pandas as pd
import matplotlib.pyplot as plt
import xlsxwriter

pd.options.mode.chained_assignment = None  # default='warn', to avoid pd error when editing with for loop, there's no for loops as of now, but it might be needed later

In [38]:
#Creation of marketwatch data links

marketwatchdata = {}

ticker=input("Insert symbol to research: ")
#For stocks on NYSE or NASDAQ just write the ticker
#For stocks on other exchanges Marketwatch does not always have data 
#If you want to find the data they might have, use 'ticker'?countrycode='countrycode'

#
urlincome = 'https://www.marketwatch.com/investing/stock/' + ticker + '/financials'               #urls are created to
urlbalance = 'https://www.marketwatch.com/investing/stock/' + ticker + '/financials/balance-sheet'#begin scraping all
urlcashflow = 'https://www.marketwatch.com/investing/stock/' + ticker + '/financials/cash-flow'   #the data
urlkeydata = 'https://www.marketwatch.com/investing/stock/' + ticker
urlprofile = 'https://www.marketwatch.com/investing/stock/' + ticker + '/company-profile'

Insert symbol to research: CSSE


In [39]:
#Scrape of marketwatch data, not a perfect source but will do for this program

income = pd.read_html(urlincome)  #see above for actual urls
balancesheet = pd.read_html(urlbalance)
cashflow = pd.read_html(urlcashflow)
keydata = pd.read_html(urlkeydata)
profile = pd.read_html(urlprofile)

In [40]:
#general formatting

profile[0] = profile[4].rename(columns={'0': 'Financial ratios'})    #lots of generic data, ratios, insider transactions
profile[1] = profile[5].rename(columns={'0': 'Revenue ratios'})      #etc. etc., needs to be formatted in a better way
profile[2] = profile[6].rename(columns={'0': 'Assets and revenues'}) #than this.
profile[3] = profile[7].rename(columns={'0': 'Margin and returns'})
profile[4] = profile[8].rename(columns={'0': 'Debt to assets'})
profile[5] = profile[9]
profile[6] = profile[10].drop(columns = ['Unnamed: 0'])

income_statement = income[4].rename(columns={'Item  Item': 'Income statement'})

balance_sheet_assets = balancesheet[4].rename(columns={'Item  Item': 'Assets'})
balance_sheet_liabilities = balancesheet[5].rename(columns={'Item  Item': 'Liabilities and Equity'})

cashflow_operations = cashflow[4].rename(columns={'Item  Item': 'Cash-flow from Operating activities'})
cashflow_investing = cashflow[5].rename(columns={'Item  Item': 'Cash-flow from Investing activities'})
cashflow_financing = cashflow[6].rename(columns={'Item  Item': 'Cash-flow from Financing activities'})

dayprice = keydata[1].rename(columns={'Close': 'Stock price'})
performance = keydata[4].rename(columns={'0': 'Performance'})
competitors = keydata[5].rename(columns={'Name': 'Competitors'})

In [41]:
#after eons and eons, I was finally able to convert the B to billions, M to millions and K to thousands, efficiently
#conversion of income statement
income_statement.iloc[:, [1,2,3,4,5]] = income_statement.iloc[:, [1,2,3,4,5]].replace({'B': '*1000000', 'M': '*1000', 'K': '', '-':'0'}, regex=True)

income_statement.iloc[:, [1,2,3,4,5]] = '=' + income_statement.iloc[:, [1,2,3,4,5]].astype(str)

#conversion of balance sheet
balance_sheet_assets.iloc[:, [1,2,3,4,5]] = balance_sheet_assets.iloc[:, [1,2,3,4,5]].replace({'B': '*1000000', 'M': '*1000', 'K': '', '-':'0'}, regex=True)
balance_sheet_liabilities.iloc[:, [1,2,3,4,5]] = balance_sheet_liabilities.iloc[:, [1,2,3,4,5]].replace({'B': '*1000000', 'M': '*1000', 'K': '', '-':'0'}, regex=True)

balance_sheet_assets.iloc[:, [1,2,3,4,5]] = '=' + balance_sheet_assets.iloc[:, [1,2,3,4,5]].astype(str)
balance_sheet_liabilities.iloc[:, [1,2,3,4,5]] = '=' + balance_sheet_liabilities.iloc[:, [1,2,3,4,5]].astype(str)

#conversion of cashflow statement
cashflow_operations.iloc[:, [1,2,3,4,5]] = cashflow_operations.iloc[:, [1,2,3,4,5]].replace({'B': '*1000000', 'M': '*1000', 'K': '', '-':'0'}, regex=True)
cashflow_investing.iloc[:, [1,2,3,4,5]] = cashflow_investing.iloc[:, [1,2,3,4,5]].replace({'B': '*1000000', 'M': '*1000', 'K': '', '-':'0'}, regex=True)
cashflow_financing.iloc[:, [1,2,3,4,5]] = cashflow_financing.iloc[:, [1,2,3,4,5]].replace({'B': '*1000000', 'M': '*1000', 'K': '', '-':'0'}, regex=True)


cashflow_operations.iloc[:, [1,2,3,4,5]] = '=' + cashflow_operations.iloc[:, [1,2,3,4,5]].astype(str)
cashflow_investing.iloc[:, [1,2,3,4,5]] = '=' + cashflow_investing.iloc[:, [1,2,3,4,5]].astype(str)
cashflow_financing.iloc[:, [1,2,3,4,5]] = '=' + cashflow_financing.iloc[:, [1,2,3,4,5]].astype(str)

#General data gets converted to num, to facilitate excel use

competitors.iloc[:,[2]] = competitors.iloc[:,[2]].replace({'B': '*1000000', 'M': '*1000', 'K': '', '-':'0', '\$':'='}, regex=True)
dayprice.iloc[:,[0]] = dayprice.iloc[:,[0]].replace({'B': '*1000000', 'M': '*1000', 'K': '', '-':'0', '\$':'='}, regex=True)

In [42]:
#all row labels are duplicated after scrape from marketwatch for some reason, was not able to deduplicate so decided to
#limit chars to half the longest title to make it more readable in excel

income_statement.iloc[:, [0]] = income_statement.iloc[:, [0]].apply(lambda x: x.str.slice(0, 36)) 

cashflow_operations.iloc[:, [0]] = cashflow_operations.iloc[:, [0]].apply(lambda x: x.str.slice(0, 39))
cashflow_investing.iloc[:, [0]] = cashflow_investing.iloc[:, [0]].apply(lambda x: x.str.slice(0, 36))
cashflow_financing.iloc[:, [0]] = cashflow_financing.iloc[:, [0]].apply(lambda x: x.str.slice(0, 38))

balance_sheet_assets.iloc[:, [0]] = balance_sheet_assets.iloc[:, [0]].apply(lambda x: x.str.slice(0, 37))
balance_sheet_liabilities.iloc[:, [0]] = balance_sheet_liabilities.iloc[:, [0]].apply(lambda x: x.str.slice(0, 61))

In [43]:
#Saving model to Excel
try:
    datawriter = pd.ExcelWriter(ticker + '.xlsx', engine='xlsxwriter')   #xlsx automatically changes to ticker.xlsx
except:
    datawriter = pd.ExcelWriter('somethingwentwrong.xlsx', engine='xlsxwriter') #don't put weird stuff as ticker, there's no error handler
#title1 = {' key stock data': str(ticker)}
#title2 = {' three statement model': str(ticker)}

#formatting
#cell_format = workbook.add_format({'font_family': 'arial'})

profile[0].to_excel(datawriter, sheet_name='Key data',startrow=2, startcol=6)
profile[1].to_excel(datawriter, sheet_name='Key data',startrow=13, startcol=1)
profile[2].to_excel(datawriter, sheet_name='Key data',startrow=19, startcol=1)
profile[3].to_excel(datawriter, sheet_name='Key data',startrow=2, startcol=10)
profile[4].to_excel(datawriter, sheet_name='Key data',startrow=2, startcol=14)
profile[5].to_excel(datawriter, sheet_name='Key data',startrow=13, startcol=6)
profile[6].to_excel(datawriter, sheet_name='Key data',startrow=13, startcol=11)

dayprice.to_excel(datawriter, sheet_name='Key data',startrow=2, startcol=1)
performance.to_excel(datawriter, sheet_name='Key data',startrow=5, startcol=1)
competitors.to_excel(datawriter, sheet_name='Key data',startrow=20, startcol=6)

income_statement.to_excel(datawriter, sheet_name='Analysis',startrow=2, startcol=1)
balance_sheet_assets.to_excel(datawriter, sheet_name='Analysis',startrow=2, startcol=9)
balance_sheet_liabilities.to_excel(datawriter, sheet_name='Analysis',startrow=42, startcol=9)

cashflow_operations.to_excel(datawriter, sheet_name='Analysis',startrow=2, startcol=17)
cashflow_investing.to_excel(datawriter, sheet_name='Analysis',startrow=24, startcol=17)
cashflow_financing.to_excel(datawriter, sheet_name='Analysis',startrow=43, startcol=17)

In [44]:
datawriter.save()